# stall library

In [ ]:
pip install awscli==1.21.12 --user
pip install pytorch_lightning
pip install pyarabic
pip install pad_sequences
pip install transformers


# library

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import re
import string
import nltk 
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
import pyarabic.araby as araby
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc
import joblib
from transformers import AutoTokenizer, AutoModel
import torch
from torch import nn,optim
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report
from tqdm.auto import tqdm
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
from torchmetrics.functional import f1_score
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
from transformers import Trainer, TrainingArguments
import random



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("your device ", device)

your device  cuda


In [ ]:
print('Make sure cuda is installed:', torch.cuda.is_available())
print('Make sure cudnn is enabled:', torch.backends.cudnn.enabled)

Make sure cuda is installed: True
Make sure cudnn is enabled: True


In [ ]:
Nile = pd.read_csv("/content/NileUniversity_Emotional (1).csv")
Nile.head(10)

,ID,text,label
0,1,الاوليمبياد الجايه هكون لسه ف الكليه ..,none
1,2,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,anger
2,3,كتنا نيله ف حظنا الهباب xD,sadness
3,4,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,joy
4,5,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,none
5,6,دلوقتي عندها اربع سنين .... ومحدش يقدر يكلمها ...,surprise
6,7,الحب يسلبها كيدها يرجعها الي براءتها الاولي وي...,love
7,8,she3er: شوفي بـ عيني امنيات الغلابه واحلام ثلث...,sympathy
8,9,86: صوتك هو حبيبي الثالث بعدك وبعد عيونك ~...,love
9,10,عمر عسر اتغلب وخرج وبيكمل مسيره مصر الفاشله في...,sadness


# PreProcessing

In [ ]:
Nile.tail()

,ID,text,label
10060,10061,2222: يلا يا جماعه حفله عمرو دياب خلصت نريح شو...,sadness
10061,10062,Mohamed5: اييييه دااا 😲😲 اوزيييل❤,surprise
10062,10063,عملتلها ريتويت بمناسبه ساره بتاعه الاوليمبياد 😃,none
10063,10064,وعليك قبلنا يانجم النجوم ياعندليب الحب والاحساس,joy
10064,10065,AlHamad يطلع ننهم كل شي سيء ووضيع كل خساسه الع...,anger


In [ ]:
Nile.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10064 entries, 0 to 10064
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10064 non-null  int64 
 1   text    10064 non-null  object
 2   label   10064 non-null  object
dtypes: int64(1), object(2)
memory usage: 314.5+ KB


# Check nulls

In [ ]:
Nile[Nile.text.isnull()]

,ID,text,label


In [ ]:
Nile = Nile.dropna()

In [ ]:
Nile.isnull().sum()

ID       0
text     0
label    0
dtype: int64

# remove Stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords_list = stopwords.words('arabic')
listToStr = ' '.join([str(elem) for elem in stopwords_list]) 
listToStr

'إذ إذا إذما إذن أف أقل أكثر ألا إلا التي الذي الذين اللاتي اللائي اللتان اللتيا اللتين اللذان اللذين اللواتي إلى إليك إليكم إليكما إليكن أم أما أما إما أن إن إنا أنا أنت أنتم أنتما أنتن إنما إنه أنى أنى آه آها أو أولاء أولئك أوه آي أي أيها إي أين أين أينما إيه بخ بس بعد بعض بك بكم بكم بكما بكن بل بلى بما بماذا بمن بنا به بها بهم بهما بهن بي بين بيد تلك تلكم تلكما ته تي تين تينك ثم ثمة حاشا حبذا حتى حيث حيثما حين خلا دون ذا ذات ذاك ذان ذانك ذلك ذلكم ذلكما ذلكن ذه ذو ذوا ذواتا ذواتي ذي ذين ذينك ريث سوف سوى شتان عدا عسى عل على عليك عليه عما عن عند غير فإذا فإن فلا فمن في فيم فيما فيه فيها قد كأن كأنما كأي كأين كذا كذلك كل كلا كلاهما كلتا كلما كليكما كليهما كم كم كما كي كيت كيف كيفما لا لاسيما لدى لست لستم لستما لستن لسن لسنا لعل لك لكم لكما لكن لكنما لكي لكيلا لم لما لن لنا له لها لهم لهما لهن لو لولا لوما لي لئن ليت ليس ليسا ليست ليستا ليسوا ما ماذا متى مذ مع مما ممن من منه منها منذ مه مهما نحن نحو نعم ها هاتان هاته هاتي هاتين هاك هاهنا هذا هذان هذه هذي هذين هكذا هل هلا هم هما هن هنا هن

In [ ]:
print(len(stopwords_list))


754


# Clean text

In [ ]:
for letter in '#.][!XR':
    Nile['text'] = Nile['text'].astype(str).str.replace(letter,'')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


# punctuation

In [ ]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

# Normalize

In [ ]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [ ]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [ ]:
def remove(string):
    return " ".join(string.split())

# Remove prefixes and suffixes

In [ ]:
st = ISRIStemmer()
def roots(text):
    return st.stem(text)

In [ ]:
def pre(text):
    return st.pre32(text)

In [ ]:
def suf(text):
    return st.suf32(text)

In [ ]:
def processPost(text): 
    # remove Negation
    text=text.replace("ﷺ", "")
    text=text.replace("لا", "")
    text=text.replace("ما", "")
    text=text.replace("لم", "")
    text=text.replace("بدون", "")
    text=text.replace("من دون", "")
    text=text.replace("لن", "")
    text=text.replace("لات", "")
   
    #Replace @username with empty string
    text = re.sub('@[^\s]+', ' ', text)
    # remove english letters
    text = re.sub('[a-zA-Z0-9_]|#|http\S+', '', text)
    
    #remove links
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
    
    #Replace #word with word
    text = re.sub(r'#([^\s]+)', r'\1', text)

    # remove punctuations
    text = remove_punctuations(text)
    
    # normalize the tweet
    text = normalize_arabic(text)
    # remove Diacritics
    text = araby.strip_diacritics(text)
    
    # remove repeated letters
    text = remove_repeating_char(text)
    # remove space
    text = remove(text)
    text = ' '.join(word for word in text.split(' ') if word not in stopwords_list)
    #remove uni code 
    text = re.sub(r'\W+', ' ', text)
    # remove suffixes
    text= suf(text)
    # remove prefixes
    text= pre(text)
    
    return text

In [ ]:
Nile["text"] = Nile['text'].apply(lambda x: processPost(x))

In [ ]:
Nile['text']

0                         اوليمبياد الجايه هكون لسه الكليه
1        عجز اوازنه وصل اتج احلي يعني لسه اقل نفلس بهاي...
2                                    كتنا نيله حظنا الهباب
3          جميعنا نريد تحقيق اهدافنا تونس تالقت حراسه ارمي
4        اوليمبياد نظامها مختلف ومواعيد اونديال مكانتش ...
                               ...                        
10060    جعه حفله عمرو دياب خلصت نريح شويه ونبدا نكتئب ...
10061                                        ايه دا اوزيل 
10062         عملتلها ريتويت بمناسبه ساره بتاعه اوليمبياد 
10063            وعليك قبا يانجم اجوم ياعندليب الحب واحساس
10064    يطلع نهم شي سيء وضيع خساسه العا تجمعت ايرانين ...
Name: text, Length: 10064, dtype: object

# Remove emoji

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
Nile["text"] = Nile["text"].apply(tokenizer.tokenize)

In [ ]:
Nile['text'] = Nile['text'].apply(lambda x: ' '.join([word for word in x ]))
Nile["text"]

0                         اوليمبياد الجايه هكون لسه الكليه
1        عجز اوازنه وصل اتج احلي يعني لسه اقل نفلس بهاي...
2                                    كتنا نيله حظنا الهباب
3          جميعنا نريد تحقيق اهدافنا تونس تالقت حراسه ارمي
4        اوليمبياد نظامها مختلف ومواعيد اونديال مكانتش ...
                               ...                        
10060    جعه حفله عمرو دياب خلصت نريح شويه ونبدا نكتئب ...
10061                                         ايه دا اوزيل
10062          عملتلها ريتويت بمناسبه ساره بتاعه اوليمبياد
10063            وعليك قبا يانجم اجوم ياعندليب الحب واحساس
10064    يطلع نهم شي سيء وضيع خساسه العا تجمعت ايرانين ...
Name: text, Length: 10064, dtype: object

In [ ]:
Nile.head(10)

,ID,text,label
0,1,اوليمبياد الجايه هكون لسه الكليه,none
1,2,عجز اوازنه وصل اتج احلي يعني لسه اقل نفلس بهاي...,anger
2,3,كتنا نيله حظنا الهباب,sadness
3,4,جميعنا نريد تحقيق اهدافنا تونس تالقت حراسه ارمي,joy
4,5,اوليمبياد نظامها مختلف ومواعيد اونديال مكانتش ...,none
5,6,دلوقتي عندها اربع سنين ومحدش يقدر يكها اله,surprise
6,7,حب يسلبها كيدها يرجعها الي براءتها اولي ويدفعه...,love
7,8,شوفي عيني امنيات الغبه واحم ثلثين الشعوب سنابي,sympathy
8,9,صوتك حبيبي الثالث بعدك وبعد عيونك بوح غزل,love
9,10,عمر عسر اتغلب وخرج وبيكمل مسيره مصر الفاشله او...,sadness


# Save cleaning_data

In [ ]:
Nile.to_csv('clean_data.csv',index=False)

# Reducing words to their roots

In [ ]:
Nile['text'] = Nile['text']. apply(lambda x: [st.stem(y) for y in x.split()])
Nile['text'] = Nile['text'].apply(lambda x: ' '.join([word for word in x ]))
Nile['text']

0                                اوليمبياد جيه هكن لسه كله
1        عجز وزن وصل اتج احل يعن لسه اقل فلس هيم لسه تب...
2                                          كتن نيل حظن هبب
3                          جمع نرد حقق هدف ونس تلق حرس ارم
4        اوليمبياد نظم خلف مواعيد ديل مكانتش قرف حجه كا...
                               ...                        
10060    جعه حفل عمرو ديب خلص نرح شوه نبد كتئب عشم نفس ...
10061                                           ايه دا وزل
10062                     عملتل ريتو نسب سره بتع اوليمبياد
10063                     وعل قبا نجم اجم ياعندليب لحب حسس
10064    طلع نهم شي سيء وضع خسس الع جمع يرن الي لفت بحر...
Name: text, Length: 10064, dtype: object

In [ ]:
Nile.to_csv('clean_data_2.csv',index=False)

# data with_out ISRIStemmer

In [ ]:
path_input = './clean_data.csv'
df = pd.read_csv(path_input)
df.head()

,ID,text,label
0,1,اوليمبياد الجايه هكون لسه الكليه,none
1,2,عجز اوازنه وصل اتج احلي يعني لسه اقل نفلس بهاي...,anger
2,3,كتنا نيله حظنا الهباب,sadness
3,4,جميعنا نريد تحقيق اهدافنا تونس تالقت حراسه ارمي,joy
4,5,اوليمبياد نظامها مختلف ومواعيد اونديال مكانتش ...,none


In [ ]:
df.dropna(inplace=True)
df.isnull().sum()

ID       0
text     0
label    0
dtype: int64

# data with ISRIStemmer

In [ ]:
path_input = './clean_data_2.csv'
dff = pd.read_csv(path_input)
dff.head()

,ID,text,label
0,1,اوليمبياد جيه هكن لسه كله,none
1,2,عجز وزن وصل اتج احل يعن لسه اقل فلس هيم لسه تب...,anger
2,3,كتن نيل حظن هبب,sadness
3,4,جمع نرد حقق هدف ونس تلق حرس ارم,joy
4,5,اوليمبياد نظم خلف مواعيد ديل مكانتش قرف حجه كا...,none


In [ ]:
dff.dropna(inplace=True)
dff.isnull().sum()

ID       0
text     0
label    0
dtype: int64

# Models without ISRIStemmer

In [ ]:
train, val = train_test_split(df[['label','text']], test_size=0.1, random_state=333)


lbl_enc = LabelEncoder()
train.loc[:,"label"] = lbl_enc.fit_transform(train["label"])
val.loc[:,"label"] = lbl_enc.transform(val["label"])

joblib.dump(lbl_enc,"label_encoder.pkl")
train.to_csv("train.csv",index=False)
val.to_csv("val.csv",index=False)

In [ ]:
train

,label,text
378,3,عظيم الحب ان تري نفسك جسد اخر
3728,6,نزلها كراك ورايا مش فاهم مشغلينها ازاي قشطه
7466,0,بعيدا اي تاثيرات شويه صعبه سليم الجبوري مراح ي...
5880,4,احنا ملتزمين الصمت اهو وقاعدين نتفرج علي اوليم...
3890,0,زهقنا اي حد بيقول معلومه وغير بيها حياه ناس وه...
...,...,...
7541,6,ازاي الواحد يقدر يحبش اوليمبياد او يحبش الرياض...
6072,6,يمكن ان تكون داخلي اكثر م
9646,3,حب يخلو اشاجرات الغياب العتب الغيره السهر الحن...
973,0,غالي متطلعش دين ابو


In [ ]:
val

,label,text
1876,4,كره طائره مصر تخسر اولي مبارياتها ام بودا اجمو...
7559,3,اشخاص يسحرون القلب الحظه اولي ويجدون الحياه جبر
1035,6,وكيف التوقيع علي ابادره الخليجيه وتسليم السلطه...
1284,5,موجوعه
4917,6,اح اله عني امهي وقد تديت ذنبي ويستر
...,...,...
4258,2,داي اسبوع التاني اوليمبياد بحبه اكتر العاب القوي
6584,6,ايه رايك اجيبلك شبكه فضه سيبك حوار الدهب ليه ا...
1108,7,نازلين نقذ مصرحرار مصر وحرائرها قلوبنا مع
6706,3,سيحدث تقاسمنا السهر عيناك ولك القصائد والقمر


In [ ]:
lbl_enc.classes_
{v: k for v, k in enumerate(lbl_enc.classes_)}

{0: 'anger',
 1: 'fear',
 2: 'joy',
 3: 'love',
 4: 'none',
 5: 'sadness',
 6: 'surprise',
 7: 'sympathy'}

In [ ]:
class ArabicDataset(Dataset):
    def __init__(self,data,max_len,model_type="Mini"):
        super().__init__()
        self.labels = data["label"].values
        self.texts = data["text"].values
        self.max_len = max_len
        model = {"Twitter": "aubmindlab/bert-base-arabertv02-twitter",
                "Base": "aubmindlab/bert-base-arabertv02",
                 "MAR" : "UBC-NLP/MARBERT"}
        self.tokenizer = AutoTokenizer.from_pretrained(model[model_type])
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self,idx):
        text = " ".join(self.texts[idx].split())
        label = self.labels[idx]
        inputs = self.tokenizer(text,padding='max_length',
                                max_length=self.max_len,truncation=True,return_tensors="pt")
        #input_ids,token_type_ids,attention_mask
        return {
            "inputs":{"input_ids":inputs["input_ids"][0],
                      "token_type_ids":inputs["token_type_ids"][0],
                      "attention_mask":inputs["attention_mask"][0],
                     },
            "labels": torch.tensor(label,dtype=torch.long) 
        }

In [ ]:
class ArabicDataModule(pl.LightningDataModule):
    def __init__(self,train_path,val_path,batch_size=12,max_len=100,model_type="Twitter"):
        super().__init__()
        self.train_path,self.val_path= train_path,val_path
        self.batch_size = batch_size
        self.max_len = max_len
        self.model_type = model_type
    
    def setup(self,stage=None):
        train = pd.read_csv(self.train_path)
        val = pd.read_csv(self.val_path)
        self.train_dataset = ArabicDataset(data=train,max_len=self.max_len,model_type=self.model_type)
        self.val_dataset = ArabicDataset(data=val,max_len=self.max_len,model_type=self.model_type)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset,batch_size=self.batch_size,shuffle=True)
    
    
    def test_dataloader(self):
        return DataLoader(self.val_dataset,batch_size=self.batch_size,shuffle=False)

In [ ]:
n_classes = 8
class ArabicBertModel(pl.LightningModule):
    def __init__(self,model_type="Mini"):
        super().__init__()
        model = {"Twitter": ("aubmindlab/bert-base-arabertv02-twitter",768),
                 "Base": ("aubmindlab/bert-base-arabertv02",768),
                 "MAR" : ("UBC-NLP/MARBERT",768)}
        self.bert_model = AutoModel.from_pretrained(model[model_type][0])
        self.fc = nn.Linear(model[model_type][1],n_classes)
    
    def forward(self,inputs):
        out = self.bert_model(**inputs)#inputs["input_ids"],inputs["token_type_ids"],inputs["attention_mask"])
        last_hidden_states = out[1]
        out = self.fc(last_hidden_states)
        return out
    
    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr=0.0001)
    
    def criterion(self,output,target):
        return nn.CrossEntropyLoss()(output,target)
    
    #TODO: adding metrics
    def training_step(self,batch,batch_idx):
        x,y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_Score = f1_score(out, y, num_classes=n_classes, average='macro')
        metrics = {"train_f1": f1_Score, "train_loss": loss}
        self.log_dict(metrics)
        return loss
    
    def validation_step(self,batch,batch_idx):
        x, y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_Score = f1_score(out, y, num_classes=n_classes, average='macro')
        metrics = {"val_f1": f1_Score, "val_loss": loss}
        self.log_dict(metrics)
        return metrics

# Arabert Twitter model

In [ ]:
# TODO: getting different models sizes results
MODEL_TYPE = "Twitter"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                batch_size=100, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_f1")]) #callbacks=[EarlyStopping(monitor="val_f1")]
trainer.fit(model,dm)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02-twitter were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.pooler.dens

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type      | Params
-----------------------------------------
0 | bert_model | BertModel | 135 M 
1 | fc         | Linear    | 6.2 K 
-----------------------------------------
135 M     Trainable params
0         Non-trainable params
135 M     Total params
540.798   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))

100%|██████████| 11/11 [00:05<00:00,  2.08it/s]

              precision    recall  f1-score   support

       anger       0.68      0.75      0.71       151
        fear       0.94      0.90      0.92       112
         joy       0.60      0.59      0.59       121
        love       0.81      0.83      0.82       115
        none       0.74      0.75      0.75       173
     sadness       0.74      0.54      0.62       120
    surprise       0.50      0.58      0.54       103
    sympathy       0.91      0.92      0.91       111

    accuracy                           0.73      1006
   macro avg       0.74      0.73      0.73      1006
weighted avg       0.74      0.73      0.73      1006



# Arabert Base Model

In [ ]:
# TODO: getting different models sizes results
MODEL_TYPE = "Base"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                batch_size=100, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_f1")]) #callbacks=[EarlyStopping(monitor="val_f1")]
trainer.fit(model,dm)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: Lig

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type      | Params
-----------------------------------------
0 | bert_model | BertModel | 135 M 
1 | fc         | Linear    | 6.2 K 
-----------------------------------------
135 M     Trainable params
0         Non-trainable params
135 M     Total params
540.798   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))


100%|██████████| 11/11 [07:54<00:00, 43.15s/it]

100%|██████████| 11/11 [00:03<00:00,  2.70it/s]

              precision    recall  f1-score   support

       anger       0.70      0.62      0.66       151
        fear       0.95      0.88      0.92       112
         joy       0.64      0.46      0.54       121
        love       0.70      0.82      0.76       115
        none       0.69      0.87      0.77       173
     sadness       0.70      0.59      0.64       120
    surprise       0.56      0.52      0.54       103
    sympathy       0.80      0.92      0.85       111

    accuracy                           0.72      1006
   macro avg       0.72      0.71      0.71      1006
weighted avg       0.71      0.72      0.71      1006



# MARBERT

In [ ]:
# TODO: getting different models sizes results
MODEL_TYPE = "MAR"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                batch_size=100, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_f1")]) #callbacks=[EarlyStopping(monitor="val_f1")]
trainer.fit(model,dm)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type      | Params
-----------------------------------------
0 | bert_model | BertModel | 162 M 
1 | fc         | Linear    | 6.2 K 
-----------------------------------------
162 M     Trainable params
0         Non-trainable params
162 M     Total params
651.390   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))

100%|██████████| 11/11 [00:03<00:00,  2.71it/s]

              precision    recall  f1-score   support

       anger       0.72      0.64      0.68       151
        fear       0.90      0.92      0.91       112
         joy       0.62      0.54      0.58       121
        love       0.74      0.77      0.76       115
        none       0.69      0.82      0.75       173
     sadness       0.58      0.64      0.61       120
    surprise       0.60      0.57      0.59       103
    sympathy       0.93      0.80      0.86       111

    accuracy                           0.72      1006
   macro avg       0.72      0.71      0.72      1006
weighted avg       0.72      0.72      0.72      1006



# Models with ISRIStemmer

In [ ]:
Nile['text']

0                                اوليمبياد جيه هكن لسه كله
1        عجز وزن وصل اتج احل يعن لسه اقل فلس هيم لسه تب...
2                                          كتن نيل حظن هبب
3                          جمع نرد حقق هدف ونس تلق حرس ارم
4        اوليمبياد نظم خلف مواعيد ديل مكانتش قرف حجه كا...
                               ...                        
10060    جعه حفل عمرو ديب خلص نرح شوه نبد كتئب عشم نفس ...
10061                                           ايه دا وزل
10062                     عملتل ريتو نسب سره بتع اوليمبياد
10063                     وعل قبا نجم اجم ياعندليب لحب حسس
10064    طلع نهم شي سيء وضع خسس الع جمع يرن الي لفت بحر...
Name: text, Length: 10064, dtype: object

In [ ]:
train, val = train_test_split(dff[['label','text']], test_size=0.1, random_state=333)


lbl_enc = LabelEncoder()
train.loc[:,"label"] = lbl_enc.fit_transform(train["label"])
val.loc[:,"label"] = lbl_enc.transform(val["label"])

joblib.dump(lbl_enc,"label_encoder.pkl")
train.to_csv("train.csv",index=False)
val.to_csv("val.csv",index=False)

In [ ]:
lbl_enc.classes_
{v: k for v, k in enumerate(lbl_enc.classes_)}

{0: 'anger',
 1: 'fear',
 2: 'joy',
 3: 'love',
 4: 'none',
 5: 'sadness',
 6: 'surprise',
 7: 'sympathy'}

In [ ]:
train

,label,text
378,3,عظم لحب ان تري نفس جسد اخر
3728,6,نزل كرك ورا مش فهم شغل ازي قشط
7466,0,بعد اي اثر شوه صعب سلم جبر راح يتر رسه
5880,4,احن لزم صمت اهو قعد فرج علي اوليمبياد لحد لتش خلص
3890,0,زهق اي حد يقل علم وغر بيه حيه ناس وهي غلط
...,...,...
7541,6,ازي وحد قدر حبش اوليمبياد او حبش ريض شكل عام
6072,6,يمكن ان تكون دخل كثر م
9646,3,حب خلو شجر غيب عتب غير سهر حنن نكه
973,0,غلي طلعش دين ابو


In [ ]:
class ArabicDataset(Dataset):
    def __init__(self,data,max_len,model_type="Mini"):
        super().__init__()
        self.labels = data["label"].values
        self.texts = data["text"].values
        self.max_len = max_len
        model = {"Twitter": "aubmindlab/bert-base-arabertv02-twitter",
                "Base": "aubmindlab/bert-base-arabertv02",
                 "MAR": "UBC-NLP/MARBERT"}
        self.tokenizer = AutoTokenizer.from_pretrained(model[model_type])
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self,idx):
        text = " ".join(self.texts[idx].split())
        label = self.labels[idx]
        inputs = self.tokenizer(text,padding='max_length',
                                max_length=self.max_len,truncation=True,return_tensors="pt")
        #input_ids,token_type_ids,attention_mask
        return {
            "inputs":{"input_ids":inputs["input_ids"][0],
                      "token_type_ids":inputs["token_type_ids"][0],
                      "attention_mask":inputs["attention_mask"][0],
                     },
            "labels": torch.tensor(label,dtype=torch.long) 
        }

In [ ]:
class ArabicDataModule(pl.LightningDataModule):
    def __init__(self,train_path,val_path,batch_size=12,max_len=100,model_type="Twitter"):
        super().__init__()
        self.train_path,self.val_path= train_path,val_path
        self.batch_size = batch_size
        self.max_len = max_len
        self.model_type = model_type
    
    def setup(self,stage=None):
        train = pd.read_csv(self.train_path)
        val = pd.read_csv(self.val_path)
        self.train_dataset = ArabicDataset(data=train,max_len=self.max_len,model_type=self.model_type)
        self.val_dataset = ArabicDataset(data=val,max_len=self.max_len,model_type=self.model_type)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset,batch_size=self.batch_size,shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,batch_size=self.batch_size,shuffle=False)
    
    def test_dataloader(self):
        return DataLoader(self.val_dataset,batch_size=self.batch_size,shuffle=False)

In [ ]:
n_classes = 8
class ArabicBertModel(pl.LightningModule):
    def __init__(self,model_type="Mini"):
        super().__init__()
        model = {"Twitter": ("aubmindlab/bert-base-arabertv02-twitter",768),
                 "Base": ("aubmindlab/bert-base-arabertv02",768),
                 "MAR": ("UBC-NLP/MARBERT",768)}
        self.bert_model = AutoModel.from_pretrained(model[model_type][0])
        self.fc = nn.Linear(model[model_type][1],n_classes)
    
    def forward(self,inputs):
        out = self.bert_model(**inputs)#inputs["input_ids"],inputs["token_type_ids"],inputs["attention_mask"])
        last_hidden_states = out[1]
        out = self.fc(last_hidden_states)
        return out
    
    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr=0.0001)
    
    def criterion(self,output,target):
        return nn.CrossEntropyLoss()(output,target)
    
    #TODO: adding metrics
    def training_step(self,batch,batch_idx):
        x,y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_Score = f1_score(out, y, num_classes=n_classes, average='macro')
        metrics = {"train_f1": f1_Score, "train_loss": loss}
        self.log_dict(metrics)
        return loss
    
    def validation_step(self,batch,batch_idx):
        x, y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_Score = f1_score(out, y, num_classes=n_classes, average='macro')
        metrics = {"val_f1": f1_Score, "val_loss": loss}
        self.log_dict(metrics)
        return metrics

# Arabert twitter model

In [ ]:
# TODO: getting different models sizes results
MODEL_TYPE = "Twitter"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                batch_size=100, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_f1")]) #callbacks=[EarlyStopping(monitor="val_f1")]
trainer.fit(model,dm)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02-twitter were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.pooler.dens

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))


100%|██████████| 11/11 [20:40<00:00, 112.76s/it]

100%|██████████| 11/11 [00:03<00:00,  2.68it/s]

              precision    recall  f1-score   support

       anger       0.63      0.51      0.56       151
        fear       0.92      0.89      0.90       112
         joy       0.58      0.40      0.48       121
        love       0.75      0.75      0.75       115
        none       0.68      0.88      0.77       173
     sadness       0.54      0.62      0.58       120
    surprise       0.38      0.37      0.38       103
    sympathy       0.88      0.90      0.89       111

    accuracy                           0.67      1006
   macro avg       0.67      0.67      0.66      1006
weighted avg       0.67      0.67      0.67      1006



# Arabert Base model

In [ ]:
# TODO: getting different models sizes results
MODEL_TYPE = "Base"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                batch_size=100, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_f1")]) #callbacks=[EarlyStopping(monitor="val_f1")]
trainer.fit(model,dm)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: Lig

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))

100%|██████████| 11/11 [00:04<00:00,  2.38it/s]

              precision    recall  f1-score   support

       anger       0.53      0.77      0.63       151
        fear       0.93      0.90      0.91       112
         joy       0.55      0.36      0.43       121
        love       0.69      0.72      0.71       115
        none       0.70      0.83      0.76       173
     sadness       0.56      0.61      0.58       120
    surprise       0.60      0.31      0.41       103
    sympathy       0.96      0.77      0.86       111

    accuracy                           0.67      1006
   macro avg       0.69      0.66      0.66      1006
weighted avg       0.68      0.67      0.67      1006



# MARBERT

In [ ]:
# TODO: getting different models sizes results
MODEL_TYPE = "MAR"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                batch_size=100, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_f1")]) #callbacks=[EarlyStopping(monitor="val_f1")]
trainer.fit(model,dm)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))


100%|██████████| 11/11 [17:36<00:00, 96.06s/it]

100%|██████████| 11/11 [00:06<00:00,  1.79it/s]

              precision    recall  f1-score   support

       anger       0.58      0.62      0.60       151
        fear       0.93      0.88      0.90       112
         joy       0.38      0.62      0.47       121
        love       0.71      0.65      0.68       115
        none       0.73      0.56      0.63       173
     sadness       0.66      0.39      0.49       120
    surprise       0.44      0.52      0.48       103
    sympathy       0.86      0.86      0.86       111

    accuracy                           0.63      1006
   macro avg       0.66      0.64      0.64      1006
weighted avg       0.66      0.63      0.64      1006



# TFIDF

In [ ]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 1),
    max_features =10000)

unigramdataGet= word_vectorizer.fit_transform(dff['text'].astype('str'))
unigramdataGet = unigramdataGet.toarray()

vocab = word_vectorizer.get_feature_names()
unigramdata_features=pd.DataFrame(np.round(unigramdataGet, 1), columns=vocab)
unigramdata_features[unigramdata_features>0] = 1

unigramdata_features.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,0في,0وك,ıs,ɺi,ρɺɹ,алепо,детеи,из,мстит,от,...,ڪم,ڪيب,ګبر,ګثر,ھم,ۆايام,ۆرق,ۆفا,یفق,۶ظم
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
pro= preprocessing.LabelEncoder()
encpro=pro.fit_transform(dff['label'])
dff['label'] = encpro
dff['label']

0        4
1        0
2        5
3        2
4        4
        ..
10059    5
10060    6
10061    4
10062    2
10063    0
Name: label, Length: 10053, dtype: int64

In [ ]:
y=dff['label']
X=unigramdata_features

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=333)

# LogisticRegression 

In [ ]:
LR= LogisticRegression(penalty = 'l2', C = 1)
LR= LR.fit(X_train , y_train)
y_pred = LR.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[180   0  23  10  16  32  33   5]
 [  3 205   5   3   3   6   3   1]
 [ 18   0 140  35  32  21  19   5]
 [  9   1  29 177   0  16   6   4]
 [  6   0  21   0 251  17  15   2]
 [ 26   0  16  18  38 111  23   8]
 [ 18   1  20   3  31  16 106   1]
 [  9   1   6   9   0  11   1 188]]


              precision    recall  f1-score   support

           0       0.67      0.60      0.63       299
           1       0.99      0.90      0.94       229
           2       0.54      0.52      0.53       270
           3       0.69      0.73      0.71       242
           4       0.68      0.80      0.73       312
           5       0.48      0.46      0.47       240
           6       0.51      0.54      0.53       196
           7       0.88      0.84      0.86       225

    accuracy                           0.67      2013
   macro avg       0.68      0.67      0.68      2013
weighted avg       0.68      0.67      0.67      2013



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


# SVM

In [ ]:
from sklearn.svm import SVC
clf=SVC(kernel='linear')
clf.fit(X_train , y_train)
y_pred =clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[160   2  28  16   8  33  31   6]
 [  4 195   3   1   6   4   4   1]
 [ 23   4 130  26  31  11  19   7]
 [ 11   2  39 151   0  18  10   6]
 [ 13   0  29   1 253  14  20   2]
 [ 45   5  28  18  34  97  20   6]
 [ 21   4  26   4  48  16  87   2]
 [  6   3   9  10   0   9   3 188]]


              precision    recall  f1-score   support

           0       0.57      0.56      0.56       284
           1       0.91      0.89      0.90       218
           2       0.45      0.52      0.48       251
           3       0.67      0.64      0.65       237
           4       0.67      0.76      0.71       332
           5       0.48      0.38      0.43       253
           6       0.45      0.42      0.43       208
           7       0.86      0.82      0.84       228

    accuracy                           0.63      2011
   macro avg       0.63      0.63      0.63      2011
weighted avg       0.63      0.63      0.63      2011



# Sequential_Minimal_Optimization

In [ ]:
X = np.array(X_train)
Y = np.array( y_train)
# Always scale the input. The most convenient way is to use a pipeline.
clf = make_pipeline(StandardScaler(),
                    SGDClassifier(max_iter=1000, tol=1e-3))
clf.fit(X, Y)


y_pred =clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[158   1  37   8  13  40  27   0]
 [  6 169   5   8   6  15   9   0]
 [ 15   1 141  24  29  23  17   1]
 [ 15   1  45 142   0  27   3   4]
 [ 20   0  57   0 210  21  23   1]
 [ 33   5  37  17  34 102  21   4]
 [ 20   1  39   4  44  17  83   0]
 [ 12   1  30   8   0  18   3 156]]


              precision    recall  f1-score   support

           0       0.57      0.56      0.56       284
           1       0.94      0.78      0.85       218
           2       0.36      0.56      0.44       251
           3       0.67      0.60      0.63       237
           4       0.62      0.63      0.63       332
           5       0.39      0.40      0.40       253
           6       0.45      0.40      0.42       208
           7       0.94      0.68      0.79       228

    accuracy                           0.58      2011
   macro avg       0.62      0.58      0.59      2011
weighted avg       0.61      0.58      0.59      2011



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:700: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


يضحك عملتلها ريتويت بمناسبه ساره بتاعه الاوليمبياد